In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('C:/Users/my.kassem/Desktop/exploratory data analysis/df_filtre_status_type_description_resolution_copy.csv')

# Afficher les premières lignes du fichier
df

,Key,Type,Summary,Description,Status,Resolution,Resolution Date,Release Note,Priority,Created,...,Components,Fix Versions,Parent Issue,Watchers,Issue Links,Subtasks,Comments,Module-Feature,Module,Feature
0,DHRD-103268,Incident,CACEIS-PROD-IN-CP - Licence expirée depuis le ...,"Bonjour à tous,\r\nNous avons CACEIS qui nous ...",Resolved,Fixed,2025-01-23T10:45:39.000+0100,NaN,Blocker,2025-01-22T13:49:58.000+0100,...,Oth/other,NaN,NaN,1,NaN,NaN,[2025-01-22T13:58:13.000+0100] Hamid Ameziani:...,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,00 - Installation & Tech Admin,Installation - Run Components
1,DHRD-103216,Story,TST - Manual override / Audit must be retested,"Due to improvements done with DHRD-100447, th...",Done,Fixed,2025-01-24T10:36:04.000+0100,NaN,Minor,2025-01-21T11:59:10.000+0100,...,NaN,NaN,NaN,0,outward: depends on DHRD-100447 [Done] - Story...,NaN,[2025-01-21T12:08:22.000+0100] Loic Brossard: ...,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,05 - Web Client,Web Navigation & Searching
2,DHRD-103195,Problem,Can't Start Workers,||Version details||Environment infos||\r\n| * ...,Resolved,Fixed,2025-01-22T16:24:24.000+0100,NaN,Blocker,2025-01-20T17:23:57.000+0100,...,NaN,NaN,NaN,0,inward: blocks DHRD-100501 [Blocked] - [DH-7.1...,NaN,[2025-01-20T18:49:05.000+0100] Abir Arrari: Tr...,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,02 - System Admin,"Scheduler, Tasks & Monitoring"
3,DHRD-103180,Bug,[AWS] S3 services failure with AWS_ROLE channel,||Version details||Environment infos||\r\n| * ...,Done,Fixed,2025-01-22T14:36:37.000+0100,NaN,Blocker,2025-01-20T16:06:24.000+0100,...,NaN,"DH-7.1.0.0, DH-7.2.0.0",NaN,0,inward: is created by DHRD-98328 [New] - [Clou...,NaN,[2025-01-20T16:23:44.000+0100] Mukesh Mishra: ...,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,16 - Cloud & DevOps,AWS Deployment
4,DHRD-103113,Bug,[Regression] : NPE raised when using the Share...,||Version details||Environment infos||\r\n|VER...,Done,Fixed,2025-01-23T09:26:04.000+0100,NaN,Blocker,2025-01-17T08:22:51.000+0100,...,NaN,"DH-7.1.0.0, DH-7.2.0.0",NaN,0,outward: includes DHRD-103220 [To be tested] -...,NaN,[2025-01-20T16:01:08.000+0100] Chaid Fatoumbi:...,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,15 - Tools,Evaluator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15976,DHRD-7282,Bug,Import error : deadlock failure,"When importing a file, the error ""* Main error...",Closed,Fixed,2012-02-02T04:00:00.000+0100,"When importing data, the multisource module ha...",NaN,2012-02-02T04:00:00.000+0100,...,C/import,"DH-3.2-00b, DH-3.3-00",NaN,0,outward: includes DHRD-21348 [Test failed] - [...,NaN,NaN,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,06 - Mapping & Connectivity,NaN
15977,DHRD-7281,Improvement,Evolutions mineures sur l'environnement des bu...,Bizness rule:\no chercher une autre bizule en ...,Closed,Fixed,2012-02-02T03:56:00.000+0100,Some bugs have been fixed and various improvem...,NaN,2012-02-02T03:56:00.000+0100,...,Cor/rule dict.,DH-3.3-00,NaN,0,outward: includes DHRD-21347 [Validated] - [DH...,NaN,NaN,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,01 - Core Admin,NaN
15978,DHRD-7280,Bug,L'icone des mails interne en attente est HS...,L'icone de boîte mail situé en bas de l'écran ...,Closed,Fixed,2012-02-02T03:53:00.000+0100,NaN,NaN,2012-02-02T03:53:00.000+0100,...,Gui/components,DH-3.3-00,NaN,0,outward: includes DHRD-21346 [Validated] - [DH...,NaN,NaN,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,04 - Rich Client,NaN
15979,DHRD-7279,Request,Ratings,Titres : \n - Les ratings côté titre sont faux...,Closed,Fixed,2012-02-01T04:13:00.000+0100,NaN,NaN,2012-02-01T04:13:00.000+0100,...,Oth/other,NaN,NaN,0,NaN,NaN,NaN,{'self': 'https://nx-jira8.my-nx.com/rest/api/...,09 - Data Enrichment,NaN


In [3]:
from sentence_transformers import SentenceTransformer


c:\Users\my.kassem\Desktop\Intelligent_analysis_of_JIRA_tickets\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import spacy
import re
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from spacy.lang.en.stop_words import STOP_WORDS as en_stopwords

In [20]:
import spacy
import re
from langdetect import detect, LangDetectException
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from spacy.lang.en.stop_words import STOP_WORDS as en_stopwords

# Charger les modèles SpaCy pour le français et l'anglais
nlp_fr = spacy.load("fr_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

# Fonction pour nettoyer le texte
def preprocess_text(text):
    # Vérifier si le texte est vide ou trop court
    if not text or len(text) < 3:
        return ""  # Retourner une chaîne vide si le texte est insuffisant pour la détection

    # 1. Retirer les caractères spéciaux et convertir en minuscule
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())  # On conserve uniquement les lettres et chiffres
    
    # 2. Détecter la langue du texte pour choisir le modèle approprié
    try:
        lang = detect(text)  # Langue détectée (fr ou en)
    except LangDetectException:
        return ""  # Si l'identification de la langue échoue, retourner une chaîne vide
    
    # Choisir le modèle SpaCy approprié
    doc = nlp_fr(text) if lang == 'fr' else nlp_en(text)
    
    # 3. Tokenisation, retrait des stopwords, et lemmatisation
    clean_tokens = []
    for token in doc:
        # Supprimer les stopwords, les symboles, et les mots avec une longueur trop courte
        if token.text not in (fr_stopwords if lang == 'fr' else en_stopwords) and len(token.text) > 1:
            clean_tokens.append(token.lemma_)  # On garde la lemmatisation du mot

    # 4. Recomposer le texte à partir des tokens traités
    return ' '.join(clean_tokens)

# Appliquer le nettoyage sur le résumé
df['clean_summary'] = df['Summary'].apply(preprocess_text)

# Vérifier les premiers résultats
print(df[['Summary', 'clean_summary']].head())


                                             Summary  \
0  CACEIS-PROD-IN-CP - Licence expirée depuis le ...   
1     TST - Manual override / Audit must be retested   
2                                Can't Start Workers   
3    [AWS] S3 services failure with AWS_ROLE channel   
4  [Regression] : NPE raised when using the Share...   

                                       clean_summary  
0              caceisprodincp licence expir 22122024  
1                  tst manual override audit reteste  
2                                   not start worker  
3             aws s3 service failure awsrole channel  
4  regression npe raise sharepoint function getsh...  


In [21]:
from sentence_transformers import SentenceTransformer

# Charger le modèle multilingue
model = SentenceTransformer("bert-base-uncased")

# Générer les embeddings pour les résumés nettoyés
embeddings = model.encode(df['clean_summary'].tolist(), show_progress_bar=True)

# Vérifier la taille des embeddings
print(f"Dimensions des embeddings : {embeddings.shape}")


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.
Batches: 100%|██████████| 500/500 [05:48<00:00,  1.43it/s]


Dimensions des embeddings : (15981, 768)


In [22]:
import hdbscan
from scipy.spatial.distance import pdist, squareform

# Calculer la matrice de distance cosinus
cosine_distances = pdist(embeddings, metric='cosine')

# Convertir la matrice de distances en matrice carrée
cosine_distance_matrix = squareform(cosine_distances)

# Appliquer HDBSCAN avec la matrice de distance cosinus
clusterer = hdbscan.HDBSCAN(min_cluster_size=20, metric='precomputed')  # Utilise 'precomputed' pour la distance pré-calculée
df['cluster'] = clusterer.fit_predict(cosine_distance_matrix)

# Afficher le nombre de tickets par cluster
print(df['cluster'].value_counts())


cluster
-1    15492
 0      422
 1       67
Name: count, dtype: int64


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fonction pour obtenir les mots-clés d'un cluster
def get_top_words(texts, n=3):
    """Retourne les 3 mots les plus représentatifs d'un groupe de textes."""
    vectorizer = TfidfVectorizer(stop_words=None)
    X = vectorizer.fit_transform(texts)
    feature_array = vectorizer.get_feature_names_out()
    tfidf_sorting = X.sum(axis=0).A1.argsort()[::-1]
    return [feature_array[i] for i in tfidf_sorting[:n]]

# Générer un label pour chaque cluster
cluster_labels = {}
for cluster_id in df['cluster'].unique():
    if cluster_id != -1:  # Ignorer les points de bruit
        texts = df[df['cluster'] == cluster_id]['clean_summary'].tolist()
        top_words = get_top_words(texts)
        cluster_labels[cluster_id] = " / ".join(top_words)

# Assigner le label de chaque cluster
df['cluster_label'] = df['cluster'].map(cluster_labels)

# Affichage des résultats
print(df[['Summary', 'cluster', 'cluster_label']].head())


                                             Summary  cluster cluster_label
0  CACEIS-PROD-IN-CP - Licence expirée depuis le ...       -1           NaN
1     TST - Manual override / Audit must be retested       -1           NaN
2                                Can't Start Workers       -1           NaN
3    [AWS] S3 services failure with AWS_ROLE channel       -1           NaN
4  [Regression] : NPE raised when using the Share...       -1           NaN


In [24]:
# Compter le nombre de tickets par cluster
cluster_counts = df["cluster"].value_counts().reset_index()
cluster_counts.columns = ["Cluster", "Nombre de Tickets"]

# Ajouter les labels des clusters
cluster_counts["Nom du Cluster"] = cluster_counts["Cluster"].map(cluster_labels)

# Trier par nombre de tickets
cluster_counts = cluster_counts.sort_values(by="Nombre de Tickets", ascending=False)

# Affichage propre
print(cluster_counts.to_string(index=False))


 Cluster  Nombre de Tickets                                               Nom du Cluster
      -1              15492                                                          NaN
       0                422 nullpointerexception / javalangnullpointerexception / import
       1                 67                            problme / rafraichissement / jour


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15981 entries, 0 to 15980
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Key              15981 non-null  object 
 1   Type             15981 non-null  object 
 2   Summary          15981 non-null  object 
 3   Description      15981 non-null  object 
 4   Status           15981 non-null  object 
 5   Resolution       15981 non-null  object 
 6   Resolution Date  15975 non-null  object 
 7   Release Note     9201 non-null   object 
 8   Priority         8850 non-null   object 
 9   Created          15981 non-null  object 
 10  Updated          15981 non-null  object 
 11  Due Date         2302 non-null   object 
 12  Reporter         15981 non-null  object 
 13  Assignee         14799 non-null  object 
 14  Labels           5286 non-null   object 
 15  Components       13246 non-null  object 
 16  Fix Versions     9459 non-null   object 
 17  Parent Issue